In [1]:
import polars as pl
from polars import col as c
import networkx as nx

In [3]:
def sum_downstream_power(col: pl.Expr, df: pl.DataFrame):
    return col.map_elements(lambda x: df.filter(c("upstream") == x)["p_line"].sum(), return_dtype=pl.Float64)

def calculate_line_power(df: pl.DataFrame):
    return (c("downstream").pipe(sum_downstream_power, df = df) + c("P"))*(1+ c("F"))

def sum_power(df: pl.DataFrame, lv: int):
    
    return df.with_columns(
        pl.when(c("lv") == lv)
        .then(calculate_line_power(df=df))
        .otherwise(c("p_line")).alias("p_line")
    )

def get_node_level(G: nx.DiGraph) -> dict:
    level_mapping: dict = {}
    for node in reversed(list(nx.topological_sort(G))):
        if not len(list(G.successors(node))):
            level_mapping[node] = 0
        else:
            level_mapping[node]= max(
                level_mapping[n] for n in G.successors(node)
            ) + 1
    return level_mapping


line_data: pl.DataFrame = pl.DataFrame({
    "downstream": [1, 2, 3, 4, 5, 6, 7, 8],
    "upstream": [None, 1, 2, 1, 4, 4, 4, 6],
    "P": [0, 1, 2, 1, 4, 3, 6, 5],
    "F": [0.0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
    "p_line": [0] * 8
})

grid= nx.DiGraph()

_ = line_data.drop_nulls(subset="upstream")\
    .with_columns(
        pl.struct(c("upstream"), c("downstream"))
        .map_elements(lambda x: grid.add_edge(x["upstream"], x["downstream"]), return_dtype=pl.Struct)
    )
level_mapping: dict = get_node_level(G=grid)
line_data = line_data.with_columns(
    c("downstream").replace_strict(level_mapping, default=None).alias("lv")
)

for i in range(line_data["lv"].max() + 1):
    line_data = sum_power(df=line_data, lv=i)

print(line_data.sort("lv"))


shape: (8, 6)
┌────────────┬──────────┬─────┬─────┬────────┬─────┐
│ downstream ┆ upstream ┆ P   ┆ F   ┆ p_line ┆ lv  │
│ ---        ┆ ---      ┆ --- ┆ --- ┆ ---    ┆ --- │
│ i64        ┆ i64      ┆ i64 ┆ f64 ┆ f64    ┆ i64 │
╞════════════╪══════════╪═════╪═════╪════════╪═════╡
│ 3          ┆ 2        ┆ 2   ┆ 0.1 ┆ 2.2    ┆ 0   │
│ 5          ┆ 4        ┆ 4   ┆ 0.1 ┆ 4.4    ┆ 0   │
│ 7          ┆ 4        ┆ 6   ┆ 0.1 ┆ 6.6    ┆ 0   │
│ 8          ┆ 6        ┆ 5   ┆ 0.1 ┆ 5.5    ┆ 0   │
│ 2          ┆ 1        ┆ 1   ┆ 0.1 ┆ 3.52   ┆ 1   │
│ 6          ┆ 4        ┆ 3   ┆ 0.1 ┆ 9.35   ┆ 1   │
│ 4          ┆ 1        ┆ 1   ┆ 0.1 ┆ 23.485 ┆ 2   │
│ 1          ┆ null     ┆ 0   ┆ 0.0 ┆ 27.005 ┆ 3   │
└────────────┴──────────┴─────┴─────┴────────┴─────┘
